# The best way to swap your allocations (graph theory for medical students)

**Problem:** Internship allocation season has come around and some people aren't happy with their results! On the forums, people are advertising swaps for hospitals to see if they can have a chance at getting into a better one. Can you find an optimal set of swaps (multi-step swaps are allowed) that makes as many people as happy as possible?

**Input:** number of hospitals, list of swaps in the format `have    want`

```
8
1 5
2 4
3 6
...
```

**Output:** the best set of swap cycles (by line number)
```
Swap these 3: 1->2->3->1
Swap these 2: 1->2->1
...
```

## Solution

Each hospital can be seen as a node on a [directed graph (or digraph)](https://mathinsight.org/definition/directed_graph#:~:text=A%20directed%20graph%20is%20graph,digraph%20or%20a%20directed%20network.). Every swap is a directed edge from `have` to `want`, so our input is an edge list.

Let's first convert the edge list to an [adjacency matrix](http://ceadserv1.nku.edu/longa//classes/mat385_resources/docs/matrix.html).

In [1]:
import numpy as np

position_count = 8
swap_sequences = [(5, 2), (5, 2), (3, 2), (4, 2), (4, 3), (2, 4), (2, 4), (2, 5), (2, 5)]

swap_matrix = np.zeros((position_count, position_count))
swap_dict = dict()
for number, swap in enumerate(swap_sequences):
    adj_matrix = np.zeros((position_count, position_count))
    have, want = swap
    if not swap_dict.get((have, want)):
        swap_dict[(have, want)] = [number]
    else:
        swap_dict[(have, want)].append(number)
    adj_matrix[have-1, want-1] = 1
    swap_matrix += adj_matrix

print(swap_matrix)
print(swap_dict)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 2. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
{(5, 2): [0, 1], (3, 2): [2], (4, 2): [3], (4, 3): [4], (2, 4): [5, 6], (2, 5): [7, 8]}


Equal and opposite swaps (e.g. 2->5, 5->2) should cancel each other out.

In [2]:
for i in range(1, position_count):
    for j in range(i):
        while swap_matrix[i,j] > 0 and swap_matrix[j,i] > 0:
            first = swap_dict[(i+1,j+1)].pop()
            second = swap_dict[(j+1,i+1)].pop()
            print("Swap these 2: {0} -> {1} -> {0}".format(first, second))
            swap_matrix[i,j] -= 1
            swap_matrix[j,i] -= 1

print(swap_matrix)

Swap these 2: 3 -> 6 -> 3
Swap these 2: 1 -> 8 -> 1
Swap these 2: 0 -> 7 -> 0
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


So that covers simple swaps... what about multi-way swaps?

## Complex (multi-way) swaps

To ensure that multi-way swaps are best covered we actually need to go back to the beginning.

First we have to segment the graph into strongly connected components using [Tarjan's algorithm](https://www.youtube.com/watch?v=wUgWX0nc4NY).

In [3]:
# runs in linear time O(E+V)
# def tarjan_scc(graph: np.array):
#     unvisited = -1

Now, within each strongly connected component, we find all cycles using [Johnson's algorithm](https://www.youtube.com/watch?v=johyrWospv0).

In [8]:
# iterative implementation of johnson's algorithm using stacks
# time complexity: O(E+V)*(c+1) (where c is the number of cycles)
# space complexity: O(E+V+s) (where s is the sum of all cycles' length)
def johnson_cycles(graph: np.array, n: int):
    can_lead_to_cycle = [False]*n
    all_cycles = list()
    trace_stack = list()
    node_stack = list()
    child_count_stack = [0]*n
    blocked_nodes = set()
    unblock_map = dict()
    # start at the lowest numbered node and go up
    for start_node in range(n):
        cycles = list()
        node_stack.insert(0, start_node)
        while node_stack:
            # get the current node
            current_node = node_stack.pop(0)
            # if this node is part of a set of children we are recursing through, count down the children
            # otherwise (i.e. 0 children), go back up
            while child_count_stack[0] == 0:
                child_count_stack.pop(0)
                removal = trace_stack.pop(0)
                if can_lead_to_cycle[removal]:
                    # if a node has been completely explored and was on a cycle, it can be unblocked
                    # since this node is now unblocked we have to check for everything else that could be unblocked
                    all_to_unblock = [removal]
                    next_unblock = unblock_map.pop(all_to_unblock[-1])
                    while next_unblock:
                        all_to_unblock.append(next_unblock)
                        next_unblock = unblock_map.pop(all_to_unblock[-1])
                    blocked_nodes -= set(all_to_unblock)
            else:
                child_count_stack[0] -= 1
            # add to trace and blocked
            trace_stack.insert(0, current_node)
            print(trace_stack)
            blocked_nodes.add(current_node)
            # add the children to the exploration stack
            children = [i for i in range(n) if graph[current_node,i]]
            # do not explore any children that
            # - form a cycle (add trace to cycles list)
            # - are blocked
            explorable_children = len(children)
            to_explore = []
            for c in children:
                if c == start_node:
                    explorable_children -= 1
                    can_lead_to_cycle[current_node] = True
                    cycles.append(trace_stack)
                elif c in blocked_nodes:
                    explorable_children -= 1
                else:
                    to_explore.append(c)
            node_stack = to_explore + node_stack
            if explorable_children < 0:
                raise ValueError("Explorable children is less than 0!")
            child_count_stack.insert(0, explorable_children)
        all_cycles += cycles
    return all_cycles

johnson_cycles(swap_matrix, position_count)

IndexError: pop from empty list

Choose the combination of cycles that will cover the most available edges (i.e. fill the most swap requests).

In [5]:
def cycle_edge_cover(graph, cycles):
    pass

This algorithm features in my new web app for swapping placements, hospitals, etc., called [SwapUs](swapus.app).

Create your own swap board and trust the algorithm to help find the best swap arrangement for the most number of people!